In [ ]:

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset


import pandas as pd
import numpy as np
from sklearn.utils import resample

from torch.nn import Module, ModuleList, BatchNorm2d, MaxPool2d, BatchNorm1d, ReLU, Softmax, CrossEntropyLoss, Sequential, Dropout, Conv2d, Linear

from brevitas.nn import QuantConv2d, QuantIdentity, QuantLinear, QuantReLU
from brevitas.core.restrict_val import RestrictValueType
from tensor_norm import TensorNorm
from common import CommonWeightQuant, CommonActQuant

In [ ]:
CNV_OUT_CH_POOL = [(21, False), (21, True), (21, False)] #[(21, False), (21, True), (21, False)]
INTERMEDIATE_FC_FEATURES = [(3549, 16), (16, 16)] #[(3549, 16), (16, 16)]
LAST_FC_IN_FEATURES = 16
LAST_FC_PER_OUT_CH_SCALING = False
POOL_SIZE = 2
KERNEL_SIZE = 6

MixPrecisionBits = 4  
ConvPrecisonBits = 4  

class CNV(Module):

    def __init__(self, num_classes, weight_bit_width, act_bit_width, in_bit_width, in_ch):
        super(CNV, self).__init__()

        self.conv_features = ModuleList()
        self.linear_features = ModuleList()

        self.conv_features.append(QuantIdentity( # for Q1.7 input format
            act_quant=CommonActQuant,
            bit_width=in_bit_width,
            min_val=- 1.0,
            max_val=1.0 - 2.0 ** (-7),
            narrow_range=True,  ###If True implements the value in a range he range for weights and biases
            #will be from -2^(N-1) + 1 to 2^(N-1), where N is the bit width. This is different from the default 
            #range of -2^(N-1) to 2^(N-1) when narrow_range is False.
            #narrow_range = True makes the hardware inference more efficient
            restrict_scaling_type=RestrictValueType.POWER_OF_TWO))

        for out_ch, is_pool_enabled in CNV_OUT_CH_POOL:
            self.conv_features.append(QuantConv2d(kernel_size=KERNEL_SIZE, in_channels=in_ch, out_channels=out_ch,
                bias=True, padding=4, weight_quant=CommonWeightQuant, weight_bit_width=ConvPrecisonBits))#made bias=False
            in_ch = out_ch
            self.conv_features.append(BatchNorm2d(in_ch, eps=1e-4))
            self.conv_features.append(QuantIdentity(act_quant=CommonActQuant,bit_width=MixPrecisionBits))#MultiThreshold123
            if is_pool_enabled:
                self.conv_features.append(MaxPool2d(kernel_size=2))

        for in_features, out_features in INTERMEDIATE_FC_FEATURES:
            self.linear_features.append(QuantLinear(in_features=in_features, out_features=out_features, bias=True,
                weight_quant=CommonWeightQuant, weight_bit_width=weight_bit_width))
            self.linear_features.append(BatchNorm1d(out_features, eps=1e-4))
            self.linear_features.append(QuantIdentity(act_quant=CommonActQuant,bit_width=act_bit_width))#MultiThreshold45

        self.linear_features.append(QuantLinear(in_features=LAST_FC_IN_FEATURES, out_features=num_classes, bias=False,
            weight_quant=CommonWeightQuant, weight_bit_width=weight_bit_width))
        self.linear_features.append(TensorNorm())
        
        for m in self.modules():
            if isinstance(m, QuantConv2d) or isinstance(m, QuantLinear):
                torch.nn.init.uniform_(m.weight.data, -1, 1)
                #print(f"Weight Data Convolution {m+1}", m.weight.data)


    def clip_weights(self, min_val, max_val):
        for mod in self.conv_features:
            if isinstance(mod, QuantConv2d):
                mod.weight.data.clamp_(min_val, max_val)
                #print(f"Weight Data Convolution {mod+1}", mod.weight.data)
        for mod in self.linear_features:
            if isinstance(mod, QuantLinear):
                mod.weight.data.clamp_(min_val, max_val)
                #print(f"Weight Data Convolution {mod+1}", mod.weight.data)

    def forward(self, x):
        #print("Data Feeded:",x)
        x = 2.0 * x - torch.tensor([1.0], device=x.device)
        #print("Data Processesd(2x-1):",x)
        for mod in self.conv_features:
            x = mod(x)
            #print(f"Data After Convolution Feature {mod+1}:",x)
        x = x.view(x.shape[0], -1)
        #print("Data After Flatten:",x)
        for mod in self.linear_features:
            x = mod(x)
            #print(f"Data After Linear Feature {mod+1}:",x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNV(num_classes=5, weight_bit_width=1, act_bit_width=1, in_bit_width=8, in_ch=1)

In [ ]:
xtrain_reshape = torch.load("xtrain25000014x14float32.pth")
ytrain_tensor = torch.load("ytrain250000int64.pth")
xval_reshape = torch.load("xval4412014x14float32.pth")
yval_tensor = torch.load("yval44120int64.pth")
xtest_reshape = torch.load("xtest4527014x14float32.pth")
ytest_tensor = torch.load("ytest45270int64.pth")

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        #self.X = torch.from_numpy(X.astype(np.float32))
        #self.y = torch.from_numpy(y.astype(np.int64))
        self.X = X.unsqueeze(1)
        self.y = y
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

In [ ]:
batch_size = 100

val_data = Data(xval_reshape, yval_tensor)
val_dataloader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)

test_data = Data(xtest_reshape, ytest_tensor)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

from DatasetSplitByRounds import split_user_datasets_by_epochs, DataB

Round_xtrain_datasets = split_user_datasets_by_epochs(xtrain_reshape,25)
Round_ytrain_datasets = split_user_datasets_by_epochs(ytrain_tensor,25)


# Initialize the cumulative dataset with the first 20 parts
cumulative_xtrain_data = []
cumulative_ytrain_data = []

# Start with the first 20 parts
for i in range(20):
    cumulative_xtrain_data.append(Round_xtrain_datasets[i])
    cumulative_ytrain_data.append(Round_ytrain_datasets[i])

# Convert the lists of lists to NumPy arrays or PyTorch tensors before passing to the Data class
Concat_xtrain_data = torch.cat(cumulative_xtrain_data, axis=0)  # Concatenate into a single array
Concat_ytrain_data = torch.cat(cumulative_ytrain_data, axis=0)  # Concatenate into a single array
train_data = Data(Concat_xtrain_data, Concat_ytrain_data)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)


In [ ]:
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

In [ ]:
def test_inference(model, test_dataloader):
    model.eval()
    Tall_true_labels = []
    Tall_predicted_labels = []

    loss, total, correct = 0.0, 0.0, 0.0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss().to(device)

    for batch_idx, (xtest, ytest) in enumerate(test_dataloader):
        xtest, ytest = xtest.to(device), ytest.to(device)
        outputs = model(xtest)
        batch_loss = criterion(outputs, ytest)
        loss += batch_loss.item()
        pred = outputs.data.argmax(1, keepdim=True)
        correct += pred.eq(ytest.data.view_as(pred)).sum()
        
        total += len(ytest)
    accuracy = 100. * correct.float() / total                   
    loss = loss/total
    return accuracy, loss 

In [ ]:
import torch
import csv
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from WeightInterpreter import WeightInterpreter

# Create CSV to save results
output_file = "Results_CL_5Rounds/Accel6_training_testing_results.csv"
header = ['Run', 'Epoch', 'Train_Loss', 'Train_Accuracy', 'Validation_Loss', 'Validation_Accuracy', 'Test_Accuracy', 'Test_Loss']

if not os.path.exists(output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

# Initialize variables
all_test_accuracies = []  # Store top-1 test accuracies for all runs
T_epochs = 10
num_epochs = 5
num_runs = 6

for run in range(1, num_runs + 1):
    #print(f"========== Run {run} ==========")
    epoch_loss = []
    batch_loss = []
    test_accuracies = []
    
    if run== 1:
        for epoch in range(T_epochs):
            model.train()
            criterion.train()

            train_correct = 0
            train_total = 0
            train_batch_loss = []

            total_batches = len(train_dataloader)
            progress_bar = tqdm(total=total_batches, desc=f"Run {run} - Epoch {epoch+1}", unit="batch", position=0, leave=True)

            for batch_idx, (xtrain, ytrain) in enumerate(train_dataloader):
                xtrain, ytrain = xtrain.to(device), ytrain.to(device)
                model_preds = model(xtrain)
                _, pred_labels = torch.max(model_preds, 1)
                train_correct += torch.sum(pred_labels == ytrain).item()
                train_total += ytrain.size(0)
                loss = criterion(model_preds, ytrain)
                train_batch_loss.append(loss.item())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                model.clip_weights(-1, 1)  # This line for DPFL

                batch_loss.append(loss.item())
                progress_bar.update(1)
                progress_bar.set_postfix(batch=batch_idx + 1, refresh=True)

            progress_bar.close()

            epoch_loss.append(sum(batch_loss) / len(batch_loss))           
            
            # Validation
            model.eval()
            criterion.eval()

            val_correct = 0
            val_total = 0
            val_batch_loss = []

            with torch.no_grad():
                for batch_idx, (xval, yval) in enumerate(val_dataloader):
                    xval, yval = xval.to(device), yval.to(device)
                    val_model_preds = model(xval)
                    val_loss = criterion(val_model_preds, yval)
                    val_batch_loss.append(val_loss.item())
                    _, val_pred_labels = torch.max(val_model_preds, 1)
                    val_correct += torch.sum(val_pred_labels == yval).item()
                    val_total += yval.size(0)

            # Calculate average training and validation metrics
            avg_train_loss = sum(train_batch_loss) / len(train_batch_loss)
            train_accuracy = train_correct / train_total if train_total > 0 else 0.0
            avg_val_loss = sum(val_batch_loss) / len(val_batch_loss)
            val_accuracy = val_correct / val_total if val_total > 0 else 0.0

            #print(f"Run {run} - Epoch {epoch + 1}")
            print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy * 100:.2f}%, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")

            # Save metrics to CSV
            with open(output_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([run, epoch + 1, avg_train_loss, train_accuracy * 100, avg_val_loss, val_accuracy * 100, None, None]) 

    else:
        cumulative_xtrain_data.append(Round_xtrain_datasets[run+18])
        cumulative_ytrain_data.append(Round_ytrain_datasets[run+18])
        Concat_xtrain_data = torch.cat(cumulative_xtrain_data, axis=0)  # Concatenate into a single array
        Concat_ytrain_data = torch.cat(cumulative_ytrain_data, axis=0)
        train_data = Data(Concat_xtrain_data, Concat_ytrain_data)
        train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            model.train()
            criterion.train()

            train_correct = 0
            train_total = 0
            train_batch_loss = []

            total_batches = len(train_dataloader)
            progress_bar = tqdm(total=total_batches, desc=f"Run {run} - Epoch {epoch+1}", unit="batch", position=0, leave=True)

            for batch_idx, (xtrain, ytrain) in enumerate(train_dataloader):
                xtrain, ytrain = xtrain.to(device), ytrain.to(device)
                model_preds = model(xtrain)
                _, pred_labels = torch.max(model_preds, 1)
                train_correct += torch.sum(pred_labels == ytrain).item()
                train_total += ytrain.size(0)
                loss = criterion(model_preds, ytrain)
                train_batch_loss.append(loss.item())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                model.clip_weights(-1, 1)  # This line for DPFL

                batch_loss.append(loss.item())
                progress_bar.update(1)
                progress_bar.set_postfix(batch=batch_idx + 1, refresh=True)

            progress_bar.close()

            epoch_loss.append(sum(batch_loss) / len(batch_loss))

            
            # Validation
            model.eval()
            criterion.eval()

            val_correct = 0
            val_total = 0
            val_batch_loss = []

            with torch.no_grad():
                for batch_idx, (xval, yval) in enumerate(val_dataloader):
                    xval, yval = xval.to(device), yval.to(device)
                    val_model_preds = model(xval)
                    val_loss = criterion(val_model_preds, yval)
                    val_batch_loss.append(val_loss.item())
                    _, val_pred_labels = torch.max(val_model_preds, 1)
                    val_correct += torch.sum(val_pred_labels == yval).item()
                    val_total += yval.size(0)

            # Calculate average training and validation metrics
            avg_train_loss = sum(train_batch_loss) / len(train_batch_loss)
            train_accuracy = train_correct / train_total if train_total > 0 else 0.0
            avg_val_loss = sum(val_batch_loss) / len(val_batch_loss)
            val_accuracy = val_correct / val_total if val_total > 0 else 0.0

            #print(f"Run {run} - Epoch {epoch + 1}")
            print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy * 100:.2f}%, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")

            # Save metrics to CSV
            with open(output_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([run, epoch + 1, avg_train_loss, train_accuracy * 100, avg_val_loss, val_accuracy * 100, None, None])
        

    torch.save(model.state_dict(), f"Results_CL_5Rounds/Accel6_R{run}_model_state.pth")     
    # Testing part
    model.eval()
    test_accuracy, test_loss = test_inference(model, test_dataloader)  # Assuming this returns a dictionary with keys 'accuracy' and 'loss'
#     test_accuracy = test_results['accuracy']
#     test_loss = test_results['loss']
    all_test_accuracies.append(test_accuracy)

    print(f"Run {run} - Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}")

    # Save testing metrics to CSV
    with open(output_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([run, None, None, None, None, None, test_accuracy, test_loss])
    WeightInterpreter(model, "Accel6")
# print(all_test_accuracies)        
# Plot Top-1 Test Accuracies
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_runs + 1), all_test_accuracies, marker='o', linestyle='-', color='b')
# plt.title("Top-1 Test Accuracy Across Runs")
plt.xlabel("Training Cycles (Each Cycle: 5 Epochs)")
plt.ylabel("Top-1 Test Accuracy (%)")
plt.grid(True)
plt.savefig("Results_CL_5Rounds/Accel6_test_accuracies_plot.png")
plt.show()
